## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/TextMining_WordEmbedding

/content/drive/MyDrive/TextMining_WordEmbedding


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net

In [4]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [7]:
len(corpus)

1020

In [8]:
corpus[:50]

['애니메이션에서나 볼 수 있었던 자동차 변신 로봇이 일본에서 실제로 만들어진다',
 "11일 소프트뱅크 계열 로봇 제조업체인 아스라테크(Asratec)와 산세이 테크놀로지, 브레이브 로보틱스가 3사 합작으로 변신 로봇 '제이 다이트 라이드(J-deite RIDE)'를 제작한다고 발표했다",
 '이 로봇은 아스라테크와 브레이브 로보틱스가 참여한 로봇 제조 계획 "프로젝트 제이 다이트\'의 일환으로 지난해 6월부터 개발이 진행되고 있다',
 "하드웨어 설계 ·제작은 브레이브 로보틱스가 맡고 제어용 소프트웨어 '브이시도(V-Sido)'의 개발은 아스라테크가 담당한다",
 '산세이 테크놀로지는 로봇의 양산화를 위해서 합류했다',
 "'제이 다이트 라이드'는 내년에 완성돼 테마 파크와 각종 이벤트 참여, 기업 홍보 등의 목적으로 쓰이게 된다",
 "이후에는 기체의 크기를 키워 2020년에는 5m에 달하는 '제이 다이트 오리지널'로 재탄생할 예정이다",
 "디자인은 애니메이션 '기동전사 건담', '달려라 번개호' 등 유명 애니메이션에 등장하는 로봇을 창조한 오오카와라 쿠니오가 맡았다",
 "오오카와라는 '제이 다이트 라이드' 외에 프로젝트에서 제작할 예정인 다른 로봇의 디자인도 맡는다",
 "'제이다이트 라이드'의 머리 부분 스케치",
 '언뜻 보기에도 건담이 연상된다',
 '이 로봇은 10초만에 자동차로 변신할 수 있으며 전체 길이는 약 4m이다',
 '로봇 모드에서는 두 다리로 걸을 수 있다',
 '차량 모드에서는 평범한 자동차와 마찬가지로 사람이 운전석에 앉아 조작하면 된다',
 "'철인28호'처럼 로봇과 떨어져 원격 조종을 해도 된다",
 '최대 2명이 탈 수 있다',
 '로봇이 두 다리로 걷는 속도는 시속 5km에 불과하지만 발 부분의 바퀴를 이용하면 한시간에 최고 30km를 갈 수 있다',
 '차로 변신했을 때 최고 속도는 시속 60km이다',
 "한편 '제이다이트 라이드'의 1.5m짜리 시제품은 14일(현지시간) 미국 올랜도에서 열리는 '국제 놀이기구 엑스

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]
조사, 구두점, 숫자가 가지는 의미가 크지 않기 때문에 제거하고 정규화하였다. 

In [58]:
okt = Okt()

In [59]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
  stopwords=['Josa','Punctuation','Number']
  tokenized=[]
  for i,j in okt.pos(sentence, stem=True, norm=True):
    if j in stopwords:
      continue
    tokenized.append(i)
  return tokenized

In [60]:
corpus[0]

'애니메이션에서나 볼 수 있었던 자동차 변신 로봇이 일본에서 실제로 만들어진다'

In [61]:
CustomTokenization(corpus[0])

['애니메이션', '볼', '수', '있다', '자동차', '변신', '로봇', '일본', '실제', '만들어지다']

In [62]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [63]:
#문장의 개수
len(corpus_tokenized)

1020

In [64]:
#총 단어 몇 개인지 확인
count=0
for i in corpus_tokenized:
  count+=len(i)
count

17656

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]
- size=200
- min_count=0 (1번만 나와도 제외되는 단어가 없도록)
- sg=1 (skip-gram)
- iter=200
- window=5

In [65]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size=200, min_count=0, sg=1, iter=200, window=5)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 3869


In [66]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 100)

(1562993, 1765600)

In [79]:
#3869개 중에 random하게 10개만 뽑아서 유사한 의미를 가진 단어들을 잘 뽑아내는지 확인해본다
import numpy as np
index=np.random.randint(0,3869,size=10,)
test_lst=[]
for i in index:
  test_lst.append(list(model_no_pretrained.wv.vocab.keys())[i])
test_lst

['깊다', '용차', '단', '뿐이다', '가능하다', '송유관', '널', '월간', '담배', '오카']

In [80]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용
no_pretrained_result=[]
for i in test_lst:
  print(i)
  print(model_no_pretrained.wv.most_similar(i))
  no_pretrained_result.append(model_no_pretrained.wv.most_similar(i))
  print('')

깊다
[('전자담배', 0.9468771815299988), ('COPD', 0.8381132483482361), ('판단', 0.7717611789703369), ('이해', 0.7635878324508667), ('악화시키다', 0.7223408818244934), ('높이', 0.7126742005348206), ('일단', 0.6921277642250061), ('정체', 0.6895352602005005), ('폐쇄성', 0.6757900714874268), ('타당', 0.6731985807418823)]

용차
[('개조', 0.8915778398513794), ('싶다', 0.7736740708351135), ('경', 0.7297766208648682), ('헬리콥터', 0.7151106595993042), ('주', 0.7034539580345154), ('소바', 0.6819175481796265), ('저지', 0.6809823513031006), ('운전면허', 0.6721454858779907), ('조화롭다', 0.6596438884735107), ('균일하다', 0.6595157384872437)]

단
[('바둑', 0.803973913192749), ('대결', 0.7977696657180786), ('조립', 0.7924191951751709), ('그다음', 0.7878753542900085), ('이세돌', 0.7860395908355713), ('되돌아오다', 0.7793856859207153), ('탈출', 0.7709308862686157), ('대의', 0.7374035120010376), ('초가', 0.7336506843566895), ('로켓', 0.7261476516723633)]

뿐이다
[('나쁘다', 0.9571444988250732), ('운', 0.8381593227386475), ('걸리다', 0.7986886501312256), ('필자', 0.7538519501686096), ('자주', 0.7

In [81]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200


In [82]:
# 모델 생성
model_pretrained = Word2Vec(size=200, min_count=0, sg=1, iter=200, window=5)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

현재 vocabulary size(V) : 3869


In [83]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31473


In [84]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 100)

(1739369, 1765600)

In [85]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# 이전에 추출된 10개의 단어에 대해서 어떻게 진행하는지 확인한다
# .wv.most_similar() 등 사용
pretrained_result=[]
for i in test_lst:
  print(i)
  print(model_no_pretrained.wv.most_similar(i))
  pretrained_result.append(model_no_pretrained.wv.most_similar(i))
  print('')

깊다
[('전자담배', 0.9468771815299988), ('COPD', 0.8381132483482361), ('판단', 0.7717611789703369), ('이해', 0.7635878324508667), ('악화시키다', 0.7223408818244934), ('높이', 0.7126742005348206), ('일단', 0.6921277642250061), ('정체', 0.6895352602005005), ('폐쇄성', 0.6757900714874268), ('타당', 0.6731985807418823)]

용차
[('개조', 0.8915778398513794), ('싶다', 0.7736740708351135), ('경', 0.7297766208648682), ('헬리콥터', 0.7151106595993042), ('주', 0.7034539580345154), ('소바', 0.6819175481796265), ('저지', 0.6809823513031006), ('운전면허', 0.6721454858779907), ('조화롭다', 0.6596438884735107), ('균일하다', 0.6595157384872437)]

단
[('바둑', 0.803973913192749), ('대결', 0.7977696657180786), ('조립', 0.7924191951751709), ('그다음', 0.7878753542900085), ('이세돌', 0.7860395908355713), ('되돌아오다', 0.7793856859207153), ('탈출', 0.7709308862686157), ('대의', 0.7374035120010376), ('초가', 0.7336506843566895), ('로켓', 0.7261476516723633)]

뿐이다
[('나쁘다', 0.9571444988250732), ('운', 0.8381593227386475), ('걸리다', 0.7986886501312256), ('필자', 0.7538519501686096), ('자주', 0.7

# 3번

In [86]:
for i in range(10):
  print(test_lst[i])
  print(no_pretrained_result[i])
  print(pretrained_result[i])
  print('')

깊다
[('전자담배', 0.9468771815299988), ('COPD', 0.8381132483482361), ('판단', 0.7717611789703369), ('이해', 0.7635878324508667), ('악화시키다', 0.7223408818244934), ('높이', 0.7126742005348206), ('일단', 0.6921277642250061), ('정체', 0.6895352602005005), ('폐쇄성', 0.6757900714874268), ('타당', 0.6731985807418823)]
[('전자담배', 0.9468771815299988), ('COPD', 0.8381132483482361), ('판단', 0.7717611789703369), ('이해', 0.7635878324508667), ('악화시키다', 0.7223408818244934), ('높이', 0.7126742005348206), ('일단', 0.6921277642250061), ('정체', 0.6895352602005005), ('폐쇄성', 0.6757900714874268), ('타당', 0.6731985807418823)]

용차
[('개조', 0.8915778398513794), ('싶다', 0.7736740708351135), ('경', 0.7297766208648682), ('헬리콥터', 0.7151106595993042), ('주', 0.7034539580345154), ('소바', 0.6819175481796265), ('저지', 0.6809823513031006), ('운전면허', 0.6721454858779907), ('조화롭다', 0.6596438884735107), ('균일하다', 0.6595157384872437)]
[('개조', 0.8915778398513794), ('싶다', 0.7736740708351135), ('경', 0.7297766208648682), ('헬리콥터', 0.7151106595993042), ('주', 0.703453

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]
- 랜덤하게 10개 단어를 추출하여 사전학습이 안된 모델과 사전학습이 되어 있는 모델 사이의 차이가 있는지 확인해보았다.
- 의미가 명확한 단어를 뽑아낸 것이 아니라 랜덤하게 추출하다보니 사전학습과 관계없이 동일한 결과가 나왔다

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]
- pca보다는 t-SNE가 유사한 의미의 단어에 대해 비슷한 위치에 더 잘 시각화한다.
- t-SNE에서 보면 (색깔, 색, 흰색, 파란색), (현재, 지금), (초등학교, 중학교) 등이 비슷한 위치에 나타난다. 하지만, 이 역시 2차원으로 축소하다보니 군집이 뭉개져 나타나서 어떤게 유사한 단어인지 찾아보기 어렵다.

In [107]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3869

In [108]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [109]:
pca = PCA(n_components=2, random_state = 0).fit_transform(vector)
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

In [110]:
print(pca.shape)
print(tsne.shape)

(3869, 2)
(3869, 2)


In [115]:
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [117]:
# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

- iter과 perplexity를 바꿔가면서 시각화 결과를 그려보았다. 
- iter, perplexity가 달라짐에 따라 눈에 띄는 변화가 나타나지는 않는다.
- iter이 증가할수록 아주 미세하게 유사한 단어끼리 더 뭉쳐서 나타난다.
- perplexity가 증가할수록 점들이 더 퍼져 나타난다. 

In [119]:
# iter=[500, 1000, 1500], perplexity=50일때 
iter_v = [500, 1000, 1500]
for i in iter_v:
    tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = i).fit_transform(vector)

    data = go.Scatter(
      x=tsne[:,0],
      y=tsne[:,1],
      text = unique_text,
      mode='markers',
      marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
    )

    layout = go.Layout(
      autosize=False,
      width=1000,
      height=1000,
      margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
      ),
    )

    fig = go.Figure(data=[data], layout=layout)
    fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [120]:
# iter=1000, perplexity=[25,50,100,150]일때 
perplexity_v = [25,50,100,150]
for i in perplexity_v:
    tsne = TSNE(n_components = 2, random_state = 0, perplexity = i, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

    data = go.Scatter(
      x=tsne[:,0],
      y=tsne[:,1],
      text = unique_text,
      mode='markers',
      marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
    )

    layout = go.Layout(
      autosize=False,
      width=1000,
      height=1000,
      margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
      ),
    )

    fig = go.Figure(data=[data], layout=layout)
    fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!